Daniel Schwartz, Rafael Bastidas, and Leo Tong

In [2]:
import http.client
import mimetypes
import SecretKey
import pandas as pd
conn = http.client.HTTPSConnection("cloud.iexapis.com")
payload = ''
headers = {
  'Cookie': 'ctoken=3d354f10fe14491bade90b88325c8622'
}
conn.request("GET", "/stable/stock/market/collection/sector/?collectionName=Retail+Trade&token=" + SecretKey.get_secret_key() + "&format=csv", payload, headers)
res = conn.getresponse()
retail_data = res.read().decode("utf-8")

f = open('retail_trade_data.csv', 'w')
f.write(str(retail_data))
f.close()
retail_trade_data = pd.read_csv("retail_trade_data.csv")
retail_trade_data.head()

,symbol,companyName,primaryExchange,calculationPrice,open,openTime,openSource,close,closeTime,closeSource,...,iexOpen,iexOpenTime,iexClose,iexCloseTime,marketCap,peRatio,week52High,week52Low,ytdChange,lastTradeTime
0,282330-KP,"BGF Retail Co., Ltd.",Korea Exchange,close,NaN,NaN,official,NaN,NaN,official,...,NaN,NaN,NaN,NaN,0.0,NaN,180500.0000,105500.0000,-0.202959,NaN
1,TTSH,"Tile Shop Holdings, Inc.",US OTC,close,4.35,1.605883e+12,official,4.500,1.605911e+12,official,...,NaN,NaN,NaN,NaN,232715250.0,555.56,4.7300,0.9000,3.179840,1.605906e+12
2,SHLDQ,Sears Holdings Corp.,US OTC,close,0.17,1.605883e+12,official,0.173,1.605911e+12,official,...,NaN,NaN,NaN,NaN,71095561.0,-0.01,0.4066,0.1205,-0.090597,1.605906e+12
3,PIR,"Pier 1 Imports, Inc.",New York Stock Exchange,close,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15158480.0,-0.05,31.0000,3.0000,-0.460606,1.581714e+12
4,LSTYF,Lifestyle China Group Ltd.,US OTC,close,NaN,NaN,official,NaN,NaN,official,...,NaN,NaN,NaN,NaN,256278750.0,-10.29,0.2500,0.1700,-0.300000,1.603460e+12


Our initial request to the API is to gather companies, and symbols from the Retail Trade Sector. We used the parameter csv to receive a text/csv response, then we created a csv file, inputted the string into the file, then read the csv file using pandas to get our dataframe. The SecretKey is a separate python file that contains the secret key.

In [3]:
retail_trade_data["primaryExchange"].value_counts()

US OTC                      661
New York Stock Exchange     102
Mexican Stock Exchange       95
NASDAQ                       90
London Stock Exchange        82
BSE Ltd.                     66
Euronext Paris               47
XETRA                        39
Korea Exchange               34
Toronto Stock Exchange       33
TSX Venture Exchange         20
Tel Aviv Stock Exchange      17
Euronext Brussels             7
Euronext Amsterdam            6
NYSE American                 5
Euronext Lisbon               3
Frankfurt Stock Exchange      2
Euronext Dublin               2
Berlin Stock Exchange         1
Name: primaryExchange, dtype: int64

The problem is here is that many of these companies are foreign so we must clean our data to include only companies that use the primary exchanges: US OTC, New York Stock Exchange, NYSE American, and NASDAQ

In [4]:
test1_data = retail_trade_data.copy()[retail_trade_data["primaryExchange"] == "US OTC"] 
test2_data = retail_trade_data.copy()[retail_trade_data["primaryExchange"] == "New York Stock Exchange"] 
test3_data = retail_trade_data.copy()[retail_trade_data["primaryExchange"] == "NASDAQ"] 
test4_data = retail_trade_data.copy()[retail_trade_data["primaryExchange"] == "NYSE American"] 
american_list = [test1_data, test2_data, test3_data, test4_data]
american_retail_data = pd.concat(american_list)
american_retail_data["primaryExchange"].value_counts()
#cleans non american companies

US OTC                     661
New York Stock Exchange    102
NASDAQ                      90
NYSE American                5
Name: primaryExchange, dtype: int64

In [5]:
american_retail_data = american_retail_data.copy()[retail_trade_data["symbol"] != "DSW"]

/Users/dan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [6]:
american_retail_data.head()

,symbol,companyName,primaryExchange,calculationPrice,open,openTime,openSource,close,closeTime,closeSource,...,iexOpen,iexOpenTime,iexClose,iexCloseTime,marketCap,peRatio,week52High,week52Low,ytdChange,lastTradeTime
1,TTSH,"Tile Shop Holdings, Inc.",US OTC,close,4.35,1.605883e+12,official,4.500,1.605911e+12,official,...,NaN,NaN,NaN,NaN,2.327152e+08,555.56,4.7300,0.9000,3.179840,1.605906e+12
2,SHLDQ,Sears Holdings Corp.,US OTC,close,0.17,1.605883e+12,official,0.173,1.605911e+12,official,...,NaN,NaN,NaN,NaN,7.109556e+07,-0.01,0.4066,0.1205,-0.090597,1.605906e+12
4,LSTYF,Lifestyle China Group Ltd.,US OTC,close,NaN,NaN,official,NaN,NaN,official,...,NaN,NaN,NaN,NaN,2.562788e+08,-10.29,0.2500,0.1700,-0.300000,1.603460e+12
5,RYKKF,"Ryohin Keikaku Co., Ltd.",US OTC,close,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.443518e+09,20.15,24.8800,16.0000,-0.265217,1.580490e+12
10,NDEND,Bankers Cobalt Corp.,US OTC,previousclose,NaN,NaN,official,NaN,NaN,official,...,NaN,NaN,NaN,NaN,1.042875e+06,-0.25,0.2880,0.0160,-0.100000,1.591882e+12


After some testing, we discovered that the response for DSW is empty, so we removed it from our data set.

In [7]:
import time
adv_retail_list = []
for company in american_retail_data["symbol"]:
    print(company)
    conn = http.client.HTTPSConnection("cloud.iexapis.com")
    payload = ''
    headers = {
      'Authorization': 'pk_f771dbbe9132446cacf7ad344c7671ac ',
      'Cookie': 'ctoken=462b015eed9841c88f080e9c4381e596'
    }
    conn.request("GET", "/stable/stock/" + company + "/advanced-stats?token=" + SecretKey.get_secret_key() + "&format=csv", payload, headers)
    res = conn.getresponse()
    company_data = res.read().decode("utf-8")
    title = company
    f = open(title + '.csv', 'w')
    f.write(str(company_data))
    f.close()
    company_data = pd.read_csv(title + ".csv")
    company_data["symbol"] = company
    adv_retail_list.append(company_data)
    time.sleep(.2)


TTSH
SHLDQ
LSTYF
RYKKF
NDEND
JSAIY
BEBE
JRONF
RTTO
HENGF
NCLTY
ASOMY
MSMKF
VEST
VLGEB
CMEIF
CKHUY
FUTL
CCKRF
AUNM
CRRFY
JEMC
BLIBQ
HENGY
ECDPD
TFACF
GPOVF
ANCE
JSTLF
JFROF
SUFF
MTRAF
ECDP
DUNBF
BOXS
PWWBF
SAKL
MCCOF
QOLCF
WGRGF
KKOYF
HSDEF
BTOOY
YGYIP
RCKZF
TVDRF
MJDS
CBKC
RLEVF
SAOGF
STZU
CPPCY
PYAT20
BELR
KGFHY
SURRF
PDNIF
CASLF
CYYHF
WTWGF
MRNJ
AHFD
SSHKF
SRHGF
SMPP
CQCQ
JAWM
JCYCF
LOVV
LHUAY
GFDGF
MLTO
FEPTY
AZRH
YAMNF
GPOOY
MONOF
RHDGF
VYND
NWRLY
YMDAF
FUJCF
CRCW
GMELF
DINOD
NXGPF
AKOM
KIDBQ
TKAYF
HDVTY
AINPF
MRPZF
PWLK
DFILF
GNC
DQJCF
WHGPF
PRGLY
CLCGY
NNWWF
NDENF
LEFUF
FHNIY
BDVSF
MRWSF
HAGN
ANCSQ
YGYI
JSNSF
GGGSF
PTMSY
LTSY
ISHI
FLHI
PGCMF
GGBBF
WUMSY
ERPSY
WOLZY
CUYTY
TLRDQ
BDVSY
AXFOF
GDIGF
BNNNF
WOSSF
IDGBF
ELPQF
ANCUF
DNOPY
FRCOY
PHDWY
CTAIF
PIRRQ
NWPN
TSSJF
TLDE
BZWHF
SIETY
MBAY
BEUHF
SNAAF
BRKO
FREDQ
WETGF
WINH
CDNTF
BBII
PLNHF
SSOK
IPLB
RBCPF
NMSEF
EPRJF
SNHR
JCYGY
CDBT
GLVNF
OSTKO
TALN
SGPMY
PDGNF
VERKF
CHNO
RALEY
SHAOF
SIGO
SMTDF
RADLY
BWXMF
PBBSF
PKPYY
SDGCF
MMRTY
HCM

This cell runs through each company in this list and sends a request for advanced data, puts the response into a csv file, reads the csv files as a dataframe, and puts the dataframe into a list.

In [8]:
adv_retail_data = pd.concat(adv_retail_list)
adv_retail_data.head()

,week52change,week52high,week52low,marketcap,employees,day200MovingAvg,day50MovingAvg,float,avg10Volume,avg30Volume,...,forwardPERatio2,revenuePerEmployee2,revenuePerShare2,profitMargin2,enterpriseValueToRevenue2,priceToSales2,beta2,enterpriseValue2,debtToEquity2,priceToBook2
0,3.128440,4.7300,0.9000,2.327152e+08,1638.0,2.2500,3.4400,29647613.0,280618.0,206720.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-0.225604,0.4066,0.1205,7.109556e+07,89000.0,0.1895,0.1920,378608931.0,221569.9,174056.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-0.351852,0.2500,0.1750,2.562788e+08,1162.0,0.2221,0.2185,367111255.0,NaN,70.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-0.039171,23.7000,8.9000,5.484073e+09,NaN,14.0100,17.3400,223211483.0,150.0,106.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-0.488636,0.2880,0.0160,1.042875e+06,NaN,0.1001,0.0638,9453697.0,200.1,1657.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
adv_retail_data.index = pd.RangeIndex(0, len(adv_retail_data.index))
adv_retail_data.head(10)

,week52change,week52high,week52low,marketcap,employees,day200MovingAvg,day50MovingAvg,float,avg10Volume,avg30Volume,...,forwardPERatio2,revenuePerEmployee2,revenuePerShare2,profitMargin2,enterpriseValueToRevenue2,priceToSales2,beta2,enterpriseValue2,debtToEquity2,priceToBook2
0,3.128440,4.7300,0.9000,2.327152e+08,1638.0,2.2500,3.4400,2.964761e+07,280618.0,206720.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.225604,0.4066,0.1205,7.109556e+07,89000.0,0.1895,0.1920,3.786089e+08,221569.9,174056.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.351852,0.2500,0.1750,2.562788e+08,1162.0,0.2221,0.2185,3.671113e+08,NaN,70.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.039171,23.7000,8.9000,5.484073e+09,NaN,14.0100,17.3400,2.232115e+08,150.0,106.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.488636,0.2880,0.0160,1.042875e+06,NaN,0.1001,0.0638,9.453697e+06,200.1,1657.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.056706,12.4500,8.6000,6.506191e+09,NaN,10.0500,10.5300,NaN,41695.5,43959.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.066667,6.0400,2.0600,6.311200e+07,NaN,4.2200,3.8700,5.864761e+06,10247.5,4596.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.021290,18.3200,16.0900,1.012532e+10,115428.0,16.8600,16.1500,2.437088e+08,NaN,200.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.001429,7.1000,6.8000,3.967415e+08,4.0,6.9600,6.9500,2.019652e+07,NaN,6.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.225000,0.0350,0.0200,1.445428e+08,NaN,0.0304,0.0306,2.419152e+09,NaN,6000.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fixes the row numbers of the dataframe.

In [12]:
%store adv_retail_data


Stored 'adv_retail_data' (DataFrame)


In [23]:
adv_retail_data.to_csv(r'/Users/dan/RetailData.csv', index = False)

In [14]:
print(adv_retail_data.loc[0])

week52change            3.12844
week52high                 4.73
week52low                   0.9
marketcap           2.32715e+08
employees                  1638
                       ...     
priceToSales2               NaN
beta2                       NaN
enterpriseValue2            NaN
debtToEquity2               NaN
priceToBook2                NaN
Name: 0, Length: 63, dtype: object


A closer look at one entity in the data frame. Some values are NA, so lets look at them.

In [15]:
adv_retail_data["profitMargin2"].value_counts()

Series([], Name: profitMargin2, dtype: int64)

It seems that some error in our csv files created 2 of some columns, but these second columns are empty. Lets remove some of them.

In [16]:
adv_retail_data.count()

week52change        602
week52high          649
week52low           649
marketcap           670
employees           510
                   ... 
priceToSales2         0
beta2                 0
enterpriseValue2      0
debtToEquity2         0
priceToBook2          0
Length: 63, dtype: int64

In [17]:
clean_retail_data = adv_retail_data.drop(columns =["revenuePerShare2", "revenuePerEmployee2", "priceToSales2","priceToBook2", "pegRatio2", "profitMargin2", "forwardPERatio2", "beta2", "debtToEquity2", "enterpriseValueToRevenue2", "enterpriseValue2"], axis=1 )


In [18]:
clean_retail_data.head()

,week52change,week52high,week52low,marketcap,employees,day200MovingAvg,day50MovingAvg,float,avg10Volume,avg30Volume,...,profitMargin,enterpriseValue,enterpriseValueToRevenue,priceToSales,priceToBook,forwardPERatio,pegRatio,peHigh,peLow,putCallRatio
0,3.128440,4.7300,0.9000,2.327152e+08,1638.0,2.2500,3.4400,29647613.0,280618.0,206720.33,...,-0.013113,4.446762e+08,1.31,0.68,1.777823,90.00,2.38,3.408324e+17,6.485183e+16,NaN
1,-0.225604,0.4066,0.1205,7.109556e+07,89000.0,0.1895,0.1920,378608931.0,221569.9,174056.83,...,-0.022931,4.112863e+09,0.25,0.00,-0.019096,-0.05,NaN,-5.183781e-01,-2.863244e-02,NaN
2,-0.351852,0.2500,0.1750,2.562788e+08,1162.0,0.2221,0.2185,367111255.0,NaN,70.00,...,-0.069362,3.834237e+08,2.20,1.43,0.198820,NaN,NaN,NaN,NaN,NaN
3,-0.039171,23.7000,8.9000,5.484073e+09,NaN,14.0100,17.3400,223211483.0,150.0,106.00,...,-0.094302,5.647220e+09,1.70,1.65,3.211487,NaN,NaN,NaN,NaN,NaN
4,-0.488636,0.2880,0.0160,1.042875e+06,NaN,0.1001,0.0638,9453697.0,200.1,1657.60,...,NaN,1.152884e+06,NaN,NaN,-0.378211,NaN,NaN,NaN,NaN,NaN


In [19]:
revenue_mean = clean_retail_data["revenue"].mean()
revenue_median = clean_retail_data["revenue"].median()
revenue_std = clean_retail_data["revenue"].std()
print("The mean is ${0:.2f}.".format(revenue_mean))
print("The median is ${0:.2f}.".format(revenue_median))
print("The standard deviation is ${0:.2f}.".format(revenue_std))

The mean is $9925759711.88.
The median is $1804594000.00.
The standard deviation is $31176322041.46.


In [20]:
clean_retail_data.count()

week52change                602
week52high                  649
week52low                   649
marketcap                   670
employees                   510
day200MovingAvg             641
day50MovingAvg              641
float                       483
avg10Volume                 518
avg30Volume                 584
ttmEPS                      662
ttmDividendRate             263
companyName                 857
sharesOutstanding           857
month1ChangePercent         834
month3ChangePercent         834
month6ChangePercent         834
ytdChangePercent            834
year1ChangePercent          834
day5ChangePercent           834
day30ChangePercent          834
maxChangePercent            780
year5ChangePercent          779
year2ChangePercent          782
nextDividendDate             71
dividendYield               243
nextEarningsDate            156
exDividendDate              392
peRatio                     610
beta                        611
symbol                      857
week52hi

In [21]:
print("max revenue: ${0:.2f}.".format(clean_retail_data["revenue"].max()))
print("min revenue: ${0:.2f}.".format(clean_retail_data["revenue"].min()))

max revenue: $523964000000.00.
min revenue: $437.00.


In [24]:
clean_retail_data.to_csv(r'/Users/dan/CleanRetailData.csv')